In [0]:
from pyspark.sql import functions as F
def create_df_from_leetcode_input(leetcode_input):
    individual_lines = leetcode_input.split('\n')
    cols = [c.strip() for c in individual_lines[0].split('|')[1:-1]]
    data = [tuple([d.strip() for d in row.split('|')[1:-1]]) for row in individual_lines[2:]]
    df = spark.createDataFrame(data,cols)
    return df

In [0]:
def check_acceptance(generated_output_df, expected_output_df):
    diff_df = expected_output_df.exceptAll(generated_output_df)
    if diff_df.count() > 0:
        print(" ⚠️ NOT ACCEPTED ⚠️ ")
        diff_df.display()
    else:
        print(" 🙌 ACCEPTED 🙌")

In [0]:
%skip
person_input = """| personId | lastName | firstName |
| -------- | -------- | --------- |
| 1        | Wang     | Allen     |
| 2        | Alice    | Bob       |"""
address_input = """| addressId | personId | city          | state      |
| --------- | -------- | ------------- | ---------- |
| 1         | 2        | New York City | New York   |
| 2         | 3        | Leetcode      | California |"""
expected_output = """| firstName | lastName | city          | state    |
| --------- | -------- | ------------- | -------- |
| Allen     | Wang     | null          | null     |
| Bob       | Alice    | New York City | New York |"""

In [0]:
%skip
person_df = create_df_from_leetcode_input(person_input)
address_df = create_df_from_leetcode_input(address_input)
expected_output_df = create_df_from_leetcode_input(expected_output)

In [0]:
%skip
print(person_df.columns)
print(address_df.columns)
print(expected_output_df.columns)

['personId', 'lastName', 'firstName']
['addressId', 'personId', 'city', 'state']
['firstName', 'lastName', 'city', 'state']


In [0]:
%skip
person_df.display()
address_df.display()
expected_output_df.display()

personId,lastName,firstName
1,Wang,Allen
2,Alice,Bob


addressId,personId,city,state
1,2,New York City,New York
2,3,Leetcode,California


firstName,lastName,city,state
Allen,Wang,null,null
Bob,Alice,New York City,New York


In [0]:
%skip
output_df = person_df.join(address_df, "personId", "left")\
    .select("firstName", "lastName",
            F.coalesce("city", F.lit("null")).alias("city"),
            F.coalesce("state", F.lit("null")).alias("state"))
output_df.display()

firstName,lastName,city,state
Allen,Wang,null,null
Bob,Alice,New York City,New York


In [0]:
%skip
check_acceptance(output_df, expected_output_df)

 🙌 ACCEPTED 🙌
